In [1]:
import numpy as np
import fitsio
import healpy as hp
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
corrData = fitsio.read('/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/valid_pixel_2048_corrections.fits')
realCorr = corrData['CORR_BALROG']
pixels = corrData['PIX_2048']


# Valid Indices
VI = np.where((realCorr < 1.5) & (realCorr > 0.7))[0]

print(len(VI) / len(realCorr))

realCorr = realCorr[VI]
pixels = pixels[VI]

spFile = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/valid_pixel_2048_conditions.fits'
spData = fitsio.read(spFile)

indSps = np.array([spData['AIRMASS_G'][VI], spData['AIRMASS_R'][VI], spData['AIRMASS_I'][VI], spData['AIRMASS_Z'][VI],
                   spData['EXP_TIME_G'][VI], spData['EXP_TIME_R'][VI], spData['EXP_TIME_I'][VI], spData['EXP_TIME_Z'][VI],
                   spData['SKY_VARIANCE_G'][VI], spData['SKY_VARIANCE_R'][VI], spData['SKY_VARIANCE_I'][VI], spData['SKY_VARIANCE_Z'][VI],
                   spData['SKYBRITE_G'][VI], spData['SKYBRITE_R'][VI], spData['SKYBRITE_I'][VI], spData['SKYBRITE_Z'][VI],
                   spData['SEEING_G'][VI], spData['SEEING_R'][VI], spData['SEEING_I'][VI], spData['SEEING_Z'][VI],
                   spData['ZPT_G'][VI], spData['ZPT_R'][VI], spData['ZPT_I'][VI], spData['ZPT_Z'][VI],
                   spData['STELLAR_DENSITY'][VI]
                   ])

0.8668883276400442


In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(25, activation='relu', input_shape = (25,)),
  tf.keras.layers.Dense(25, activation='relu'),
  tf.keras.layers.Dense(25, activation='relu'),
  tf.keras.layers.Dense(1)
])

2022-08-05 11:11:02.137746: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
realCorr = realCorr - np.average(realCorr)
realCorr= realCorr / np.std(realCorr)

for sp in indSps:
    sp = sp - np.average(sp)
    sp = sp / np.std(sp)

In [5]:
trainIndices = np.full(len(indSps[0]), True, dtype = bool)
for i in range(len(trainIndices)):
    if np.random.rand() > 0.90:
        trainIndices[i] = False

xTrain = indSps[:,trainIndices].T
yTrain = np.array(realCorr[trainIndices])

In [6]:
# binLims = np.linspace(0.7, 1.5, 9)
# bins = []
# for i in range(len(binLims) - 1):
#     bins.append(np.where((yTrain >= binLims[i]) & (yTrain < binLims[i + 1]))[0])
    
# bins = np.array(bins, dtype = object)

In [7]:
# maxBin = 0
# for i in bins:
#     if len(i) > maxBin:
#         maxBin = len(i)

In [8]:
xTest = indSps[:,~trainIndices].T
yTest = np.array(realCorr[~trainIndices])

In [9]:
# xTrainAdj = []
# yTrainAdj = []

# for i in range(len(bins)):
#     counts = 0
#     while counts < maxBin:
#         for j in range(len(bins[i])):
#             if counts == maxBin:
#                 break
#             xTrainAdj.append(xTrain[bins[i][j]])
#             yTrainAdj.append(yTrain[bins[i][j]])
#             counts += 1
            
# xTrainAdj = np.array(xTrainAdj)
# yTrainAdj = np.array(yTrainAdj)

In [10]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x = xTrain, y = yTrain, epochs = 100)

Epoch 1/100
19275/19275 [==============================] - 520s 27ms/step - loss: 227.2938
Epoch 2/100
19275/19275 [==============================] - 165s 9ms/step - loss: 1.1241
Epoch 3/100
19275/19275 [==============================] - 145s 7ms/step - loss: 1.0006
Epoch 4/100
19275/19275 [==============================] - 304s 16ms/step - loss: 1.0000
Epoch 5/100
 1186/19275 [>.............................] - ETA: 6:45 - loss: 0.9922

KeyboardInterrupt: 

In [ ]:
yPredTest = model(xTest)

In [ ]:
plt.figure(dpi = 150)
plt.scatter(yTest, yPredTest, s = 0.5)
plt.plot([np.min(yTest), np.max(yTest)], [np.min(yTest), np.max(yTest)], color = 'k', linestyle = '--')
plt.xlabel('Real Correction Value')
plt.ylabel('Calc Correction Value') 
plt.show()

In [ ]:
yPredTrain = model(xTrain)

In [ ]:
plt.figure(dpi = 150)
plt.scatter(yTrain, yPredTrain, s = 0.5)
plt.plot([np.min(yTrain), np.max(yTrain)], [np.min(yTrain), np.max(yTrain)], color = 'k', linestyle = '--')
plt.xlabel('Real Correction Value')
plt.ylabel('Calc Correction Value')
plt.show()